<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/CIFAR10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

성능
- 54% 정확도
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [38]:
import torch
import torchvision
import torchvision.transforms as transforms

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [40]:
learning_rate = 0.001
training_epoch = 2
batch_size = 2

In [41]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [42]:
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
      super(Net,self).__init__()

      self.conv1 = torch.nn.Conv2d(3,6,5)
      self.pool = torch.nn.MaxPool2d(2,2)
      self.conv2 = torch.nn.Conv2d(6,16,5)
      self.fc1 = torch.nn.Linear(16*5*5,120)
      self.fc2 = torch.nn.Linear(120,84)
      self.fc3 = torch.nn.Linear(84,10)

    def forward(self,x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1,16*5*5)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

net = Net().to(device)

In [43]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(),lr=learning_rate,momentum=0.9)


In [44]:
for epoch in range(training_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
       
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.258
[1,  4000] loss: 1.958
[1,  6000] loss: 1.783
[1,  8000] loss: 1.704
[1, 10000] loss: 1.621
[1, 12000] loss: 1.601
[1, 14000] loss: 1.589
[1, 16000] loss: 1.540
[1, 18000] loss: 1.512
[1, 20000] loss: 1.477
[1, 22000] loss: 1.483
[1, 24000] loss: 1.445
[2,  2000] loss: 1.415
[2,  4000] loss: 1.383
[2,  6000] loss: 1.404
[2,  8000] loss: 1.355
[2, 10000] loss: 1.376
[2, 12000] loss: 1.368
[2, 14000] loss: 1.366
[2, 16000] loss: 1.349
[2, 18000] loss: 1.364
[2, 20000] loss: 1.324
[2, 22000] loss: 1.316
[2, 24000] loss: 1.333
Finished Training


In [45]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images,labels = data

        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f17abafe588>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f17abafe588>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    self._shutdown_workers()
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
  File "/usr/local/l

Accuracy of the network on the 10000 test images: 51 %


In [46]:
# accuracy : 51%